**Transfer Learning:** Using a pre-trained model like **ResNet50** leverages knowledge learned from a massive dataset **(ImageNet)**. This is extremely beneficial, especially when you have limited fMRI data. Transfer learning often leads to better performance and faster training. As we train the model we will explore the other options for training.


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

**Global Average Pooling:** The GlobalAveragePooling2D layer reduces the spatial dimensions of the feature maps to a single vector, which is then fed into the fully connected layers. This helps prevent overfitting and reduces the number of parameters to train.
**Binary Classification Output:** The final Dense layer with a sigmoid activation is appropriate for binary classification problems.
**Adam Optimizer:** Adam is a popular and effective optimizer for deep learning.

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Load pre-trained ResNet-50 model. For the input shape we will adjust it according to the dataset we will get. If we are using functional connectivity matrices, we might consider using 2D convolutional layers to directly process the matrix data.

In [ ]:
# ... (Load and preprocess your data: train_data, train_labels, val_data, val_labels, test_data, test_labels) ...

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(a,b,c))

Freeze initial layers.
Freezing layers in transfer learning allows you to leverage pre-trained knowledge, prevent overfitting, speed up training, and fine-tune specific features for your task.

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

Determine the output tensor of the frozen base model. This is usually the last layer of the base model.

In [ ]:
x = base_model.output

A common practice is to add a GlobalAveragePooling2D layer after the base model's output. This layer averages the feature maps across all spatial dimensions, resulting in a single vector for each image. This helps reduce the number of parameters and can improve generalization.

In [ ]:
x = GlobalAveragePooling2D()(x)

Add one or more fully connected (Dense) layers after the global average pooling layer. These layers will learn to combine the features extracted by the base model for your specific classification task.

In [ ]:
# Batch Normalization (helps stabilize training)
x = BatchNormalization()(x)

In [ ]:
# Dropout (reduces overfitting)
x = Dropout(0.5)(x)  # You can adjust the dropout rate (0.2 - 0.5 is common)

In [ ]:
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
#you can adjust the number of neurons

Another Batch Normalization and Dropout can be added if needed

*   x = BatchNormalization()(x)
*   x = Dropout(a)(x)



The final layer should be a dense layer with a sigmoid activation function for binary classification (or softmax for multi-class classification). The number of neurons in this layer should match the number of classes in your problem (1 for binary, more for multi-class).

In [ ]:
predictions = Dense(1, activation='sigmoid')(x)  # Output layer (1 neuron for binary, sigmoid activation)

Combine the base model and the custom layers you've added to create the final model.

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

Train the Model:

In [ ]:
 training = model.fit(
    train_data, train_labels,
    validation_data=(val_data, val_labels),
    epochs="a",
    batch_size="b"
)

Evaluate the Model:

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Get predictions on the test set
y_pred_proba = model.predict(test_data)  # Get probabilities
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

Calculate additional metrics

In [ ]:
accuracy = accuracy_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
auc_roc = roc_auc_score(test_labels, y_pred_proba)

In [ ]:
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")
print(f"Test AUC-ROC: {auc_roc:.4f}")